In [1]:
import numpy as np
import pandas as pd
from SERGIO.sergio import sergio
from SERGIO.utils import convert_interaction_net_to_networkx, file_reader,run_sergio_1ct
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv
import warnings
from scipy.stats import ttest_rel
import random
import itertools
import networkx as nx

In [2]:
path = 'data_sets/De-noised_100G_9T_300cPerT_4_DS1/'
input_file_targets=  path + 'Interaction_cID_4.txt'
input_file_regs= path + 'Regs_cID_4.txt'

In [3]:
input_regs = file_reader(input_file_regs)
input_targets = file_reader(input_file_targets)
input_regs = np.asarray(input_regs,dtype = float)


In [4]:
def run_sergio_1ct(input_regs,input_targets,number_sc = 100):
    '''Run Sergio for one cell type'''
    n_MR, a = np.shape(input_regs)
    n_celltypes= a-1
    id_cell_type = 0
    warnings.warn('Input containing '+str(n_celltypes)+' cell types, considering only cell type '+str(id_cell_type))
    n_celltypes = 1
    input_regs = np.array(input_regs)[:,id_cell_type:1+id_cell_type+1]
    n_reg_genes = len(input_targets)
    sim = sergio(number_genes=n_MR+n_reg_genes, number_bins = n_celltypes,number_sc =number_sc , noise_params = 1, decays=0.8, sampling_state=15, noise_type='dpd')
    sim.build_graph_array(input_taregts = input_targets, input_regs= input_regs, shared_coop_state=2)
    sim.simulate()
    expr = sim.getExpressions()
    expr_clean = np.concatenate(expr, axis = 1)
    return expr_clean

In [5]:
def networkx_2_input_reg(G,hill_coef = 2):
    data = []
    for i,k in G.in_degree():
        if k>0:
            to,_,c = list(zip(*G.in_edges(i,data = True)))
            inter = [temp['weight'] for temp in c]
            G.in_edges(i,data = True)
            data+=[list(itertools.chain.from_iterable([[i],[k],to,inter,[hill_coef]]))]
    return data


In [6]:
def create_random_graph(N):
    '''N:number of nodes
    Note that the number of nodes in the graph will likely be smaller than N
    Weights sampled from 2 uniform distributions
    '''
    weight_pos = 3
    weight_neg = -weight_pos
    band=1
    frac_pos = 0.5 #fraction of positive links over total
    G = nx.generators.random_k_out_graph(n = 100,k = 4,alpha = 0.9)
    largest = max(nx.strongly_connected_components(G), key=len)
    G = G.subgraph(largest).copy()#filter graph to maximum  SCC
    G = nx.DiGraph(G)#remove multi links
    M = len(G.edges())#n. of links
    w1 = np.random.uniform(weight_neg-band,weight_neg+band,int(M*frac_pos))#sample weights from 2 uniform distributions
    w2 = np.random.uniform(weight_pos-band,weight_pos+band,M-len(w1))
    w = np.concatenate([w1, w2])
    random.shuffle(w)#avoid having correlation in the position
    J = nx.adjacency_matrix(G)
    J.data = w
    G = nx.from_scipy_sparse_matrix(J,create_using=nx.DiGraph())
    return G
G = create_random_graph(50)

In [7]:
input_targets = networkx_2_input_reg(G,hill_coef = 2)

In [ ]:
#input_regs = [[len(input_targets),0]]  runs a simulation with one master regulator disconnetted to anything,
wt = run_sergio_1ct(input_regs = [[len(input_targets),0]],input_targets=input_targets,number_sc=100)#this is a workaround to run a simulation without MR, 

<ipython-input-4-fc6f6cc42388>:6: UserWarning: Input containing 1 cell types, considering only cell type 0
  warnings.warn('Input containing '+str(n_celltypes)+' cell types, considering only cell type '+str(id_cell_type))


In [38]:
a,b = zip(*list(G.in_degree()))
np.array(b)

array([ 4,  8,  4, 18,  3,  3,  3,  2, 13,  5,  2,  2,  7,  3,  9, 10,  4,
        2,  2,  2, 12,  6,  6,  1,  1,  6,  5,  3,  5,  1,  1,  3,  4,  1,
        4,  2,  4,  4,  3,  6,  5,  2,  5, 12,  1,  4,  3,  1,  7,  2,  3,
        3,  4,  3,  3,  2,  3,  2,  2,  2,  2,  2,  2,  2,  3,  1,  2,  1,
        6,  3,  1,  1,  1])

In [40]:
for indRow, row in enumerate(data):
    nRegs = np.int(np.float(row[1]))
    for regId, K, in zip(row[2: 2 + nRegs], row[2+nRegs : 2+2*nRegs]):
        print(regId)
        

24
48
79
98
0
6
19
42
80
86
87
98
0
13
42
64
0
12
15
19
27
32
34
42
44
47
52
57
74
75
81
83
85
86
0
14
42
32
63
83
1
53
68
1
61
1
6
7
14
28
39
40
52
55
57
62
84
87
15
36
71
81
97
2
96
2
20
2
15
45
57
66
77
78
2
31
47
29
33
54
61
70
76
83
85
90
4
9
13
18
21
29
38
52
68
70
4
51
55
75
4
20
4
28
54
64
6
20
24
28
35
40
59
62
63
64
67
70
6
9
50
61
64
79
7
36
60
66
91
99
40
9
9
18
24
35
47
50
32
44
68
74
89
12
34
81
12
48
60
61
97
12
71
13
29
98
13
36
39
63
99
14
53
79
85
15
47
45
48
77
91
17
53
59
77
17
67
86
17
18
33
50
59
77
17
31
63
66
74
18
75
38
90
91
96
99
19
38
39
51
59
60
62
74
76
80
87
91
34
20
66
76
99
21
80
89
21
21
31
35
48
73
84
97
24
96
36
68
90
27
55
87
27
44
73
80
27
33
67
28
54
97
45
84
29
33
98
31
39
76
81
34
73
35
38
44
51
45
67
75
89
53
70
78
50
78
85
52
54
57
78
83
84
96
55
71
89
86
73
79


<ipython-input-40-6182162a9a61>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nRegs = np.int(np.float(row[1]))
<ipython-input-40-6182162a9a61>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nRegs = np.int(np.float(row